## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("../Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns = ['EIN','NAME'])

In [3]:
# Determine the number of unique values in each column.
unique_values = application_df.dtypes[application_df.dtypes == 'object'].index.tolist()
application_df[unique_values].nunique()

APPLICATION_TYPE          17
AFFILIATION                6
CLASSIFICATION            71
USE_CASE                   5
ORGANIZATION               4
INCOME_AMT                 9
SPECIAL_CONSIDERATIONS     2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
application_type = application_df['APPLICATION_TYPE'].value_counts()
application_type

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T14        3
T25        3
T15        2
T29        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [7]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = application_type[application_type < 1000].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
Other     2266
T4        1542
T6        1216
T5        1173
T19       1065
Name: APPLICATION_TYPE, dtype: int64

In [8]:
# Look at CLASSIFICATION value counts for binning
classification_counts = application_df['CLASSIFICATION'].value_counts()
classification_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C1820        1
C2570        1
C3700        1
C2600        1
C1732        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [10]:
# You may find it helpful to look at CLASSIFICATION value counts >1
for c,n in classification_counts.items():
    if n>1:
        print(c,n)

C1000 17326
C2000 6074
C1200 4837
C3000 1918
C2100 1883
C7000 777
C1700 287
C4000 194
C5000 116
C1270 114
C2700 104
C2800 95
C7100 75
C1300 58
C1280 50
C1230 36
C1400 34
C7200 32
C2300 32
C1240 30
C8000 20
C7120 18
C1500 16
C1800 15
C6000 15
C1250 14
C8200 11
C1238 10
C1278 10
C1237 9
C1235 9
C7210 7
C4100 6
C2400 6
C1720 6
C1257 5
C1600 5
C2710 3
C0 3
C1260 3
C3200 2
C1256 2
C1234 2
C1246 2
C1267 2


In [11]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = classification_counts[classification_counts < 1000].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [12]:
# Convert categorical data to numeric with `pd.get_dummies`
X_dummies=pd.get_dummies(application_df)


In [13]:
# Split our preprocessed data into our features and target arrays
y = X_dummies['IS_SUCCESSFUL'].values
X = X_dummies.drop(columns = ['IS_SUCCESSFUL']).values
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [14]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [15]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
num_input = len(X_train_scaled[0])
num_first = len(X_train_scaled[0])*2
num_second = 50

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=num_first, input_dim = num_input, activation ='relu'))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units = num_second, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units = 1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 80)                3280      
_________________________________________________________________
dense_1 (Dense)              (None, 50)                4050      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 51        
Total params: 7,381
Trainable params: 7,381
Non-trainable params: 0
_________________________________________________________________


In [16]:
# Compile the model
nn.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [17]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=200)

Epoch 1/200
804/804 [==============================] - 1s 653us/step - loss: 0.5723 - accuracy: 0.7207
Epoch 2/200
804/804 [==============================] - 1s 743us/step - loss: 0.5591 - accuracy: 0.7275
Epoch 3/200
804/804 [==============================] - 1s 777us/step - loss: 0.5556 - accuracy: 0.7296
Epoch 4/200
804/804 [==============================] - 1s 742us/step - loss: 0.5551 - accuracy: 0.7292
Epoch 5/200
804/804 [==============================] - 1s 728us/step - loss: 0.5538 - accuracy: 0.7302
Epoch 6/200
804/804 [==============================] - 1s 668us/step - loss: 0.5518 - accuracy: 0.7311
Epoch 7/200
804/804 [==============================] - 1s 643us/step - loss: 0.5517 - accuracy: 0.7322
Epoch 8/200
804/804 [==============================] - 1s 681us/step - loss: 0.5509 - accuracy: 0.7325
Epoch 9/200
804/804 [==============================] - 1s 702us/step - loss: 0.5500 - accuracy: 0.7322
Epoch 10/200
804/804 [==============================] - 1s 693us/step - l

804/804 [==============================] - 1s 764us/step - loss: 0.5384 - accuracy: 0.7391
Epoch 80/200
804/804 [==============================] - 1s 803us/step - loss: 0.5382 - accuracy: 0.7385
Epoch 81/200
804/804 [==============================] - 1s 721us/step - loss: 0.5385 - accuracy: 0.7395
Epoch 82/200
804/804 [==============================] - 1s 745us/step - loss: 0.5383 - accuracy: 0.7387
Epoch 83/200
804/804 [==============================] - 1s 764us/step - loss: 0.5389 - accuracy: 0.7389
Epoch 84/200
804/804 [==============================] - 1s 756us/step - loss: 0.5382 - accuracy: 0.7391
Epoch 85/200
804/804 [==============================] - 1s 718us/step - loss: 0.5387 - accuracy: 0.7387
Epoch 86/200
804/804 [==============================] - 1s 716us/step - loss: 0.5381 - accuracy: 0.7386
Epoch 87/200
804/804 [==============================] - 1s 710us/step - loss: 0.5379 - accuracy: 0.7389
Epoch 88/200
804/804 [==============================] - 1s 702us/step - loss:

804/804 [==============================] - 1s 700us/step - loss: 0.5362 - accuracy: 0.7399
Epoch 158/200
804/804 [==============================] - 1s 698us/step - loss: 0.5359 - accuracy: 0.7399
Epoch 159/200
804/804 [==============================] - 1s 718us/step - loss: 0.5362 - accuracy: 0.7402
Epoch 160/200
804/804 [==============================] - 1s 731us/step - loss: 0.5359 - accuracy: 0.7402
Epoch 161/200
804/804 [==============================] - 1s 704us/step - loss: 0.5356 - accuracy: 0.7402
Epoch 162/200
804/804 [==============================] - 1s 704us/step - loss: 0.5366 - accuracy: 0.7397
Epoch 163/200
804/804 [==============================] - 1s 689us/step - loss: 0.5360 - accuracy: 0.7400
Epoch 164/200
804/804 [==============================] - 1s 693us/step - loss: 0.5356 - accuracy: 0.7402
Epoch 165/200
804/804 [==============================] - 1s 715us/step - loss: 0.5357 - accuracy: 0.7403
Epoch 166/200
804/804 [==============================] - 1s 694us/ste

In [18]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5813 - accuracy: 0.7306
Loss: 0.5813384652137756, Accuracy: 0.7306122183799744


In [19]:
num_input = len(X_train_scaled[0])
num_first = len(X_train_scaled[0])*2
num_second = 50

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=num_first, input_dim = num_input, activation ='relu'))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units = num_second, activation='relu'))

nn.add(tf.keras.layers.Dense(units = num_second, activation='relu'))

nn.add(tf.keras.layers.Dense(units = num_second, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units = 1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 80)                3280      
_________________________________________________________________
dense_4 (Dense)              (None, 50)                4050      
_________________________________________________________________
dense_5 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_6 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 51        
Total params: 12,481
Trainable params: 12,481
Non-trainable params: 0
_________________________________________________________________


In [20]:
nn.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [21]:
fit_model = nn.fit(X_train_scaled, y_train, epochs=200)

Epoch 1/200
804/804 [==============================] - 1s 804us/step - loss: 0.5736 - accuracy: 0.7216
Epoch 2/200
804/804 [==============================] - 1s 856us/step - loss: 0.5578 - accuracy: 0.7287
Epoch 3/200
804/804 [==============================] - 1s 767us/step - loss: 0.5553 - accuracy: 0.7294
Epoch 4/200
804/804 [==============================] - 1s 812us/step - loss: 0.5536 - accuracy: 0.73040s - loss: 0.5545 - accura
Epoch 5/200
804/804 [==============================] - 1s 782us/step - loss: 0.5523 - accuracy: 0.7324
Epoch 6/200
804/804 [==============================] - 1s 786us/step - loss: 0.5518 - accuracy: 0.7317
Epoch 7/200
804/804 [==============================] - 1s 792us/step - loss: 0.5510 - accuracy: 0.7322
Epoch 8/200
804/804 [==============================] - 1s 783us/step - loss: 0.5505 - accuracy: 0.7321
Epoch 9/200
804/804 [==============================] - 1s 807us/step - loss: 0.5490 - accuracy: 0.7323
Epoch 10/200
804/804 [=========================

804/804 [==============================] - 1s 847us/step - loss: 0.5372 - accuracy: 0.7395
Epoch 80/200
804/804 [==============================] - 1s 847us/step - loss: 0.5374 - accuracy: 0.7397
Epoch 81/200
804/804 [==============================] - 1s 844us/step - loss: 0.5373 - accuracy: 0.7395
Epoch 82/200
804/804 [==============================] - 1s 846us/step - loss: 0.5372 - accuracy: 0.7395
Epoch 83/200
804/804 [==============================] - 1s 861us/step - loss: 0.5379 - accuracy: 0.73940s - loss: 0.5385 - accuracy: 
Epoch 84/200
804/804 [==============================] - 1s 834us/step - loss: 0.5372 - accuracy: 0.7392
Epoch 85/200
804/804 [==============================] - 1s 838us/step - loss: 0.5369 - accuracy: 0.7399
Epoch 86/200
804/804 [==============================] - 1s 850us/step - loss: 0.5369 - accuracy: 0.7390
Epoch 87/200
804/804 [==============================] - 1s 846us/step - loss: 0.5369 - accuracy: 0.7393
Epoch 88/200
804/804 [=========================

804/804 [==============================] - 1s 840us/step - loss: 0.5349 - accuracy: 0.74050s - loss: 0
Epoch 156/200
804/804 [==============================] - 1s 854us/step - loss: 0.5355 - accuracy: 0.7405
Epoch 157/200
804/804 [==============================] - 1s 855us/step - loss: 0.5352 - accuracy: 0.7407
Epoch 158/200
804/804 [==============================] - 1s 833us/step - loss: 0.5358 - accuracy: 0.7404
Epoch 159/200
804/804 [==============================] - 1s 836us/step - loss: 0.5355 - accuracy: 0.7403
Epoch 160/200
804/804 [==============================] - 1s 856us/step - loss: 0.5355 - accuracy: 0.7406
Epoch 161/200
804/804 [==============================] - 1s 848us/step - loss: 0.5354 - accuracy: 0.7398
Epoch 162/200
804/804 [==============================] - 1s 844us/step - loss: 0.5345 - accuracy: 0.74020s - loss: 0.5338 - accuracy: 0.
Epoch 163/200
804/804 [==============================] - 1s 873us/step - loss: 0.5367 - accuracy: 0.7402
Epoch 164/200
804/804 [==

In [22]:
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.6057 - accuracy: 0.7299
Loss: 0.60565185546875, Accuracy: 0.729912519454956


In [23]:
num_input = len(X_train_scaled[0])
num_first = len(X_train_scaled[0])*2
num_second = 50

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=num_first, input_dim = num_input, activation ='relu'))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units = num_second, activation='relu'))

nn.add(tf.keras.layers.Dense(units = num_second, activation='sigmoid'))

nn.add(tf.keras.layers.Dense(units = num_second, activation='sigmoid'))

# Output layer
nn.add(tf.keras.layers.Dense(units = 1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 80)                3280      
_________________________________________________________________
dense_9 (Dense)              (None, 50)                4050      
_________________________________________________________________
dense_10 (Dense)             (None, 50)                2550      
_________________________________________________________________
dense_11 (Dense)             (None, 50)                2550      
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 51        
Total params: 12,481
Trainable params: 12,481
Non-trainable params: 0
_________________________________________________________________


In [25]:
nn.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [26]:
fit_model = nn.fit(X_train_scaled, y_train, epochs=200)

Epoch 1/200
804/804 [==============================] - 1s 791us/step - loss: 0.5771 - accuracy: 0.7132
Epoch 2/200
804/804 [==============================] - 1s 897us/step - loss: 0.5591 - accuracy: 0.7288
Epoch 3/200
804/804 [==============================] - 1s 806us/step - loss: 0.5562 - accuracy: 0.7300
Epoch 4/200
804/804 [==============================] - 1s 823us/step - loss: 0.5544 - accuracy: 0.7306
Epoch 5/200
804/804 [==============================] - 1s 784us/step - loss: 0.5532 - accuracy: 0.7315
Epoch 6/200
804/804 [==============================] - 1s 823us/step - loss: 0.5518 - accuracy: 0.7313
Epoch 7/200
804/804 [==============================] - 1s 840us/step - loss: 0.5511 - accuracy: 0.7313
Epoch 8/200
804/804 [==============================] - 1s 806us/step - loss: 0.5511 - accuracy: 0.7307
Epoch 9/200
804/804 [==============================] - 1s 795us/step - loss: 0.5503 - accuracy: 0.7325
Epoch 10/200
804/804 [==============================] - 1s 793us/step - l

804/804 [==============================] - 1s 844us/step - loss: 0.5358 - accuracy: 0.7396
Epoch 80/200
804/804 [==============================] - 1s 909us/step - loss: 0.5359 - accuracy: 0.7394
Epoch 81/200
804/804 [==============================] - 1s 859us/step - loss: 0.5354 - accuracy: 0.73920s - loss: 0.5294 
Epoch 82/200
804/804 [==============================] - 1s 850us/step - loss: 0.5354 - accuracy: 0.7396
Epoch 83/200
804/804 [==============================] - 1s 845us/step - loss: 0.5361 - accuracy: 0.7398
Epoch 84/200
804/804 [==============================] - 1s 851us/step - loss: 0.5355 - accuracy: 0.73990s - loss: 0.5350 - accuracy
Epoch 85/200
804/804 [==============================] - 1s 858us/step - loss: 0.5355 - accuracy: 0.7402
Epoch 86/200
804/804 [==============================] - 1s 841us/step - loss: 0.5357 - accuracy: 0.7399
Epoch 87/200
804/804 [==============================] - 1s 886us/step - loss: 0.5354 - accuracy: 0.7398
Epoch 88/200
804/804 [=========

804/804 [==============================] - 1s 850us/step - loss: 0.5341 - accuracy: 0.7395
Epoch 157/200
804/804 [==============================] - 1s 852us/step - loss: 0.5347 - accuracy: 0.7395
Epoch 158/200
804/804 [==============================] - 1s 838us/step - loss: 0.5338 - accuracy: 0.7407
Epoch 159/200
804/804 [==============================] - 1s 861us/step - loss: 0.5348 - accuracy: 0.7391
Epoch 160/200
804/804 [==============================] - 1s 857us/step - loss: 0.5345 - accuracy: 0.7406
Epoch 161/200
804/804 [==============================] - 1s 868us/step - loss: 0.5339 - accuracy: 0.7404
Epoch 162/200
804/804 [==============================] - 1s 855us/step - loss: 0.5341 - accuracy: 0.7404
Epoch 163/200
804/804 [==============================] - 1s 858us/step - loss: 0.5345 - accuracy: 0.7407
Epoch 164/200
804/804 [==============================] - 1s 871us/step - loss: 0.5335 - accuracy: 0.7407
Epoch 165/200
804/804 [==============================] - 1s 844us/ste

In [27]:
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5817 - accuracy: 0.7285
Loss: 0.5816740393638611, Accuracy: 0.7285131216049194


In [29]:
# Export our model to HDF5 file
nn.save_weights('model.hdf5')